In [14]:
from dotenv import load_dotenv, find_dotenv
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType, initialize_agent
from langchain.tools import BaseTool


class SquareTool(BaseTool):
    name: str = "Multiply tool"
    description: str = "Calculates the square of a single integer input"
    def _run(self, query: int ) -> int:
        if isinstance(query, str):
            try:
                print(f"Query: {query}")
                query = int(query.split("*")[0].strip())
            except ValueError as e:
                raise ValueError("Invalid input format. Expected an integer or a valid expression.") from e
        else:
            print(f"Query: {query}")
        return query * query

tools = [SquareTool()]
llm = ChatOpenAI(model="gpt-4-turbo")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages= True)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

#print(agent.agent.llm_chain.prompt.messages)

agent.run("what is the capital of  hungary")
agent.run("what is their famous vegetarian food")
agent.run("provide the recipe")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "The capital of Hungary is Budapest."
}
```

> Finished chain.


> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "A famous vegetarian dish from Hungary is Lecsó, a vegetable stew that features bell peppers, tomato, onion, and paprika. It is often enjoyed by vegetarians and can be served with bread or as a side dish."
}
```

> Finished chain.


> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Lecsó is a traditional Hungarian vegetable stew that is simple and delicious. Here's a basic recipe:\n\nIngredients:\n- 3-4 bell peppers, sliced\n- 2 large tomatoes, chopped\n- 1 large onion, chopped\n- 2 cloves of garlic, minced\n- 2 tablespoons of vegetable oil\n- 1 teaspoon of paprika\n- Salt and pepper to taste\n\nInstructions:\n1. Heat the oil in a large pan over medium heat.\n2. Add the chopped o

"Lecsó is a traditional Hungarian vegetable stew that is simple and delicious. Here's a basic recipe:\n\nIngredients:\n- 3-4 bell peppers, sliced\n- 2 large tomatoes, chopped\n- 1 large onion, chopped\n- 2 cloves of garlic, minced\n- 2 tablespoons of vegetable oil\n- 1 teaspoon of paprika\n- Salt and pepper to taste\n\nInstructions:\n1. Heat the oil in a large pan over medium heat.\n2. Add the chopped onion and minced garlic to the pan. Sauté until the onion becomes translucent.\n3. Add the paprika to the onions and garlic, stirring quickly to prevent burning.\n4. Add the sliced bell peppers and continue to sauté for a few more minutes until they start to soften.\n5. Add the chopped tomatoes and season with salt and pepper. Stir well.\n6. Reduce the heat to low, cover the pan, and let the mixture simmer for about 20-30 minutes, stirring occasionally.\n7. Adjust seasoning as needed, and serve hot, either as a main dish with bread or as a side dish."

In [ ]:
from langchain.agents import AgentType, initialize_agent
from dotenv import load_dotenv, find_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.utilities import GoogleSearchAPIWrapper, WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.tools import Tool
from langchain_core.callbacks.base import BaseCallbackHandler
from typing import List, Dict, Any
import os

print(load_dotenv(find_dotenv()))
#print("GOOGLE_API_KEY:", os.getenv("GOOGLE_API_KEY"))

search = GoogleSearchAPIWrapper()
wikipedia = WikipediaAPIWrapper()
llm = ChatOpenAI(model="gpt-4o")

class ToolTracker(BaseCallbackHandler):
    def __init__(self):
        self.tool_usage = []
        # List of actions that aren't tools
        self.non_tool_actions = ["Final Answer"]

    def on_agent_action(self, action: Any, **kwargs: Any) -> Any:
        """Called when agent takes an action"""
        # Extract tool name from action
        if hasattr(action, 'tool'):
            tool_name = action.tool
        elif isinstance(action, dict) and "action" in action:
            tool_name = action["action"]
        else:
            tool_name = str(action)

        # Only track if it's a real tool (not in non_tool_actions)
        if tool_name not in self.non_tool_actions and tool_name not in self.tool_usage:
            self.tool_usage.append(tool_name)
            print(f"\nTool used: {tool_name}")

    def get_tools_used(self) -> List[str]:
        return self.tool_usage

    def reset(self):
        self.tool_usage = []

tools = [
    Tool(
        name="Google Search",
        description="Useful for when you need to search for current or specific information on the internet",
        func= search.run
    ),
    Tool(
        name="Wikipedia",
        func=wikipedia.run,
        description="Useful for when you need detailed background information about a topic"
    ),
    Tool(
        name="Python Calculator",
        func=PythonREPLTool().run,
        description="Useful for performing mathematical calculations or running Python code"
    ),
    Tool(
        name="Text Analyzer",
        func=lambda x: {
            'word_count': len(x.split()),
            'char_count': len(x),
            'uppercase_count': sum(1 for c in x if c.isupper())
        },
        description="Useful for analyzing text properties like word count, character count, etc."
    )            

]

memory = ConversationBufferMemory(
    memory_key = "chat_history",
    return_messages=True
)

callback_handler = ToolTracker()

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    callbacks=[callback_handler]
)

def process_query(query: str) -> Dict:
    """
    Process a user query and demonstrate dynamic tool selection
    """
    try:
        callback_handler.reset()
        result = agent.run(input=query)
        tools_used = callback_handler.get_tools_used()

        return {
            "status": "success",
            "result": result,
            "tools_used": tools_used,
            "tools_count": len(tools_used)
        }
    except Exception as e:
        return {
            "status": "error",
            "error": str(e)
        }

example_queries = [
        "What is the capital of France and what's its current population?",
        "What is the most famous food in it ?",
        "Calculate the compound interest on $1000 at 5% for 3 years",
        "How many words are in the phrase 'The quick brown fox jumps over the lazy dog'?",
        "Who invented the telephone and when was the first successful call made?"
]


for query in example_queries:
    print(f"\nQuery: {query}")
    result = process_query(query)
    print("Result:", result)

True
GOOGLE_API_KEY: 

Query: What is the capital of France and what's its current population?


> Entering new AgentExecutor chain...

Tool used: Google Search
```json
{
    "action": "Google Search",
    "action_input": "current population of Paris France"
}
```
Observation: With an official estimated population of 2,102,650 residents in January 2023 in an area of more than 105 km2 (41 sq mi), Paris is the fourth-largest city in the ... Jul 4, 2024 ... ... Recent StatisticsPopular Statistics · Most used social networks 2024 ... Estimated population of Paris in France from 1989 to 2023 (in million ... The current population of France is 66,583,249 as of Saturday, November 2, 2024, based on Worldometer's elaboration of the latest United Nations data1. · France ... ... demographic situation in France. Total population · Projections. Population structure. The population of France by sex and age is estimated by INSEE each year. Paris, city and capital of France, situated in the north-cent

In [31]:
from langchain.agents import AgentType, initialize_agent
from dotenv import load_dotenv, find_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.utilities import GoogleSearchAPIWrapper, WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.tools import Tool
from langchain_core.callbacks.base import BaseCallbackHandler
from typing import List, Dict, Any
import os

class ResourceSearchTracker(BaseCallbackHandler):
    def __init__(self):
        self.discovery_phase = []  # Track initial resource discovery
        self.detailed_searches = []  # Track detailed information gathering
        self.findings = {}  # Organized findings
        
    def on_agent_action(self, action: Any, **kwargs: Any) -> Any:
        if isinstance(action, dict):
            action_type = action.get("action", "")
            action_input = action.get("action_input", "")
            thought = kwargs.get("thought", "")
            
            # Track the search and its context
            search_info = {
                "type": action_type,
                "query": action_input,
                "thought": thought,
                "phase": "discovery" if not self.discovery_phase else "detailed"
            }
            
            if search_info["phase"] == "discovery":
                self.discovery_phase.append(search_info)
            else:
                self.detailed_searches.append(search_info)
    
    def get_findings(self) -> Dict:
        return {
            "discovery": self.discovery_phase,
            "detailed": self.detailed_searches,
            "organized_findings": self.findings
        }
    
    def reset(self):
        self.__init__()

def create_discovery_prompt(query: str) -> str:
    return f"""
    You are an AI education specialist. For this query: "{query}"
    
    Follow these EXACT steps:

    1. First Search: "Top rated machine learning courses 2024 reviews rankings"
        - List actual course names
        - Note specific ratings/rankings
        - Record time commitments
        - List actual costs

    2. Second Search: "Best GenAI courses and specializations 2024 reviews"
        - Find specific program names
        - Note duration and prerequisites
        - List concrete costs
        - Include student reviews

    3. Third Search: "Most recommended AI/ML learning platforms 2024 comparison"
        - Compare platform features
        - List pricing structures
        - Note course quality metrics
        - Include completion rates

    For EACH resource found, you MUST include:
    - Exact course/resource name
    - Actual cost (not "varies" or "depends")
    - Specific time commitment (hours/weeks)
    - Prerequisites
    - Learning format (video/interactive/text)
    - Student completion rates if available
    - Recent review scores
    
    Do NOT proceed to final answer until you have concrete details for at least:
    - 3 university courses
    - 3 MOOC specializations
    - 3 learning platforms
    - 2 comprehensive learning paths

    DO NOT provide general statements or vague information. Only include resources where you found specific, verifiable details.
    """

def create_detailed_investigation_prompt(resource: str) -> str:
    return f"""
    Let's gather detailed information about {resource} which was recommended in reviews.
    
    Find specific details about:
    1. Current availability and format
    2. Actual time commitment required
    3. Real prerequisites
    4. Updated cost
    5. Recent reviews and experiences
    
    Focus on CURRENT information (2024).
    Verify details from official sources where possible.
    Note any discrepancies or changes from previous years.
    """

class ResourceDiscoveryAgent:
    def __init__(self):
        self.search = GoogleSearchAPIWrapper()
        self.wiki = WikipediaAPIWrapper()
        self.tracker = ResourceSearchTracker()
        
        self.tools = [
            Tool(
                name="Google Search",
                description="Search for current course recommendations, reviews, and detailed information",
                func=self.search.run
            ),
            Tool(
                name="Wikipedia",
                description="Get background information about educational platforms and concepts",
                func=self.wiki.run
            )
        ]
        
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        
        self.llm = ChatOpenAI(temperature=0.7, model="gpt-4")
        
        self.agent = initialize_agent(
            self.tools,
            self.llm,
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            memory=self.memory,
            verbose=True,
            callbacks=[self.tracker]
        )

    def discover_resources(self, query: str) -> Dict[str, Any]:
        """Two-phase resource discovery process"""
        self.tracker.reset()
        
        # Phase 1: Discovery
        discovery_results = self.agent.run(
            input=create_discovery_prompt(query)
        )
        
        # Extract recommended resources from discovery phase
        recommended_resources = self._extract_recommendations(discovery_results)
        
        # Phase 2: Detailed Investigation
        for resource in recommended_resources:
            detailed_results = self.agent.run(
                input=create_detailed_investigation_prompt(resource)
            )
            self._process_detailed_results(resource, detailed_results)
        
        return self.tracker.get_findings()

    def _extract_recommendations(self, discovery_results: str) -> List[str]:
        """Extract specific resources that were recommended during discovery"""
        # This would use the LLM to parse the discovery results and identify
        # specific resources that were highly recommended
        prompt = f"""
        From these search results, extract the specific resources that were 
        consistently recommended or highly rated. Include only resources that 
        had positive reviews or recommendations:

        {discovery_results}

        Format each resource as: "Resource Name - Provider"
        """
        try:
            extraction_result = self.llm.predict(prompt)
            # Process and return as list
            return [line.strip() for line in extraction_result.split('\n') if line.strip()]
        except Exception as e:
            print(f"Error extracting recommendations: {e}")
            return []

    def _process_detailed_results(self, resource: str, details: str):
        """Process and organize detailed findings about each resource"""
        self.tracker.findings[resource] = details

def print_resource_findings(findings: Dict[str, Any]):
    """Print findings in a clear, organized format"""
    print("\n=== AI/ML LEARNING RESOURCES 2024 ===\n")
    
    print("DISCOVERY PROCESS:")
    print("-" * 40)
    for search in findings["discovery"]:
        print(f"\nSearch: {search['query']}")
        print(f"Reasoning: {search['thought']}")
    
    print("\nDETAILED FINDINGS:")
    print("-" * 40)
    for resource, details in findings["organized_findings"].items():
        print(f"\nRESOURCE: {resource}")
        print(f"Details: {details}")
        print("-" * 20)

# Usage
if __name__ == "__main__":
    print(load_dotenv(find_dotenv()))
    
    agent = ResourceDiscoveryAgent()
    query = "best resources for learning ML and GenAI in 2024"
    findings = agent.discover_resources(query)
    print_resource_findings(findings)
 

True


> Entering new AgentExecutor chain...
```json
{
    "action": "Google Search",
    "action_input": "Top rated machine learning courses 2024 reviews rankings"
}
```
Observation: May 25, 2024 ... I'm also considering a career change to AI or machine learning. If you have any recommendations for courses, schools, or career paths that could ... Jan 25, 2017 ... ... highest rated and most reviewed courses of the ones considered. It ... 10 Must-Know Machine Learning Algorithms for Data Scientists ... Apr 26, 2022 ... Coursera's Machine Learning course by Andrew Ng: coursera.org. Fast.ai's Practical Deep Learning for Coders course: course.fast.ai. edX's ... Feb 23, 2023 ... So I created a review-driven guide that recommended the best courses ... Every single Machine Learning course on the internet, ranked by your ... Jun 2, 2023 ... The Hundred-Page Machine Learning Book. Machine Learning Engineering. both by Andriy Burkov. Machine Learning is more than just training models. Apr 29, 20


--- Starting Agent Process ---


> Entering new AgentExecutor chain...
I need to find information about the best rated machine learning courses in 2024. The best way to get this information is through a Google search.
Action: Google Search
Action Input: Top rated machine learning courses in 2024
Observation: Apr 26, 2022 ... Comments Section · Skillpro's Machine Learning course by by Juan Galvan: skillpro.io · Coursera's Machine Learning course by Andrew Ng: coursera. 1 Machine Learning — Coursera ... This is the course for which all other machine learning courses are judged. This beginner's course is taught and created by ... Jun 2, 2023 ... Top 5 Machine Learning Books for 2024. Hands-On ML with Scikit-Learn ... BEST AI COURSES · Best Programming Books For Beginners · Best ... Learn Machine Learning or improve your skills online today. Choose from a wide range of Machine Learning courses offered from top universities and industry ... Jan 5, 2024 ... r/learnmachinelearning - Top AI l

In [36]:
from langchain.agents import initialize_agent, AgentType, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.tools import Tool

# Set up the search tool
search_tool = GoogleSearchAPIWrapper()

# Set up the memory buffer for context retention
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Initialize the language model
llm = ChatOpenAI(temperature=0.5, model="gpt-4")

# Define the tools that the agent will use
tools = [
    Tool(
        name="Google Search",
        func=search_tool.run,
        description="A tool to search for course details, reviews, costs, and other educational resources."
    )
]

# Custom prompt that instructs the agent to keep exploring until it reaches a thorough answer
custom_prompt = """
You are an AI assistant tasked with providing comprehensive answers. After each search,
evaluate if your answer covers all critical aspects of the query (e.g., cost, duration, reviews).
If any key detail is missing, use the tools available to gather more information or ask follow-up questions.

Continue iterating until you have a complete and detailed response. Use follow-up searches as needed.
"""

# Initialize the agent with enhanced reasoning and a custom prompt
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,  # To show the agent's reasoning steps
    agent_prompt_template=custom_prompt  # Guide the agent's behavior with this prompt
)

def run_agent_interactive(query: str, max_iterations: int = 3):
    """
    Run the agent interactively with a custom prompt that encourages iterative reasoning.
    The agent continues until a complete answer is built or max_iterations is reached.
    """
    for i in range(max_iterations):
        print(f"\n--- Iteration {i + 1} ---")
        result = agent.run(input=query)
        
        # Check if the agent indicates that more information is needed
        if "more information needed" in result.lower() or "further details required" in result.lower():
            print("Agent identified that more details are needed. Continuing with follow-up...")
        else:
            print("\n--- Final Output ---")
            print(result)
            break  # Exit loop if the agent believes it has a complete answer

# Execute the function with an initial query
if __name__ == "__main__":
    initial_query = "Top rated GEN Ai courses in 2024"
    run_agent_interactive(initial_query)



--- Iteration 1 ---


> Entering new AgentExecutor chain...
I need to search for the top-rated GEN Ai courses in 2024.
Action: Google Search
Action Input: "Top rated GEN Ai courses in 2024"
Observation: Dec 3, 2023 ... Well, there are quite a lot of college courses from Yahn Lecun New York or Stanford and the others. Fully connected, ConvNet, LSTM (they are ... Generative AI Courses Online. Discover Generative AI (GenAI) courses that focus on skills like model creation, natural language processing, and AI applications. Mar 17, 2024 ... I am looking for a certificate/degree in the field of AI to help me transition to an AI/Machine Learning career, any recommendations? MIT? Explore AI courses that cover skills like machine learning, data analysis, and neural networks. Build expertise for careers in AI research, robotics, and data ... May 25, 2024 ... I scanned for AI courses and enrolled in the most popular according to the platform's reviews, then ranked them based on factors like audi